In [43]:
import pysam
import pandas as pd
import pybedtools
from io import StringIO
import pybedtools 
import numpy as np
pybedtools.helpers.set_tempdir("/data/projects/temp")

In [49]:
reference_file_path= "/data/projects/Resources/Gencode_genome_annotation/GRCh38.primary_assembly.genome.fa"
reference_fasta = pysam.FastaFile(reference_file_path)

In [3]:
vcf_file_path = "/data/private/pdutta/Collab_data/Oliver_data/New_data/background_boundaries.SpliceAI.vcf.gz"
vcf_file = pysam.VariantFile(vcf_file_path)

[E::idx_find_and_load] Could not retrieve index file for '/data/private/pdutta/Collab_data/Oliver_data/New_data/background_boundaries.SpliceAI.vcf.gz'


In [4]:
Splice_VCF = "/data/private/pdutta/Collab_data/Oliver_data/New_data/sigma_boundaries.SpliceAI.vcf.gz"
splice_vcf_file = pysam.VariantFile(Splice_VCF)

[E::idx_find_and_load] Could not retrieve index file for '/data/private/pdutta/Collab_data/Oliver_data/New_data/sigma_boundaries.SpliceAI.vcf.gz'


In [11]:
df_acceptor = pd.read_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Acceptor_raw_files/Acceptor_80BP_bed.tsv", sep="\t")
df_acceptor

,transcript_id,exon_id,transcript_type,strand,chr_name,acceptor_40BP,start,end
0,ENST00000003583.12,ENSE00003692979.1,protein_coding,-,chr1,GTGAAATGATCTCTACAAATGCTGTGTTTTGCTGTTTCAGGTGCTT...,24401417,24401496
1,ENST00000003583.12,ENSE00003638911.1,protein_coding,-,chr1,TTGAAGTATCTCGATGACACCACCTTCTCTTCCCTTTCAGAATGAT...,24383964,24384043
2,ENST00000003583.12,ENSE00003688858.1,protein_coding,-,chr1,TTATGTGACAATGCTGATTCCTTTGGTTGTCTCCTTACAGTGCGCC...,24379784,24379863
3,ENST00000003583.12,ENSE00003631141.1,protein_coding,-,chr1,GAAAGGTACTGAGTACATTGGGTGTATTTTCTCCCCCTAGGCCTCT...,24373771,24373850
4,ENST00000003583.12,ENSE00003516207.1,protein_coding,-,chr1,AAGAGTTCCTTATTCTGTCCTAAAATTTCTTTCTCCTTAGGGCATT...,24369800,24369879
...,...,...,...,...,...,...,...,...
1255761,ENST00000684226.1,ENSE00003767626.1,non_stop_decay,-,chrY,ACTTTGGGTTACTTTATGACTGTTGCTTTTTTCCTTCTAGCCTGCC...,13297809,13297888
1255762,ENST00000684226.1,ENSE00003761778.1,non_stop_decay,-,chrY,TTCAGTAGTTACTCTCACAAATAGTTTTTTCCCACAATAGGTATTG...,13260365,13260444
1255763,ENST00000684226.1,ENSE00001713248.1,non_stop_decay,-,chrY,ACTGTCCAATACTTTTTGTTTATTTATTTATTTAATATAGGTGGAG...,13251148,13251227
1255764,ENST00000684226.1,ENSE00003917271.1,non_stop_decay,-,chrY,TCCTGCAAACCCCAATGTGAACTTGTGGTGCCTTTTCCAGGCCTCC...,13234787,13234866


In [12]:
str_io_acceptor = StringIO()
df_acceptor[['chr_name','start','end','transcript_id','strand']].to_csv(str_io_acceptor, sep='\t', index=False, header=False)
str_io_acceptor.seek(0)
acceptor_bed = pybedtools.BedTool(str_io_acceptor.read(), from_string=True)

In [31]:
# List to store BED entries
bed_entries = []

# Iterate over each record in the VCF file and create BED entries
for rec in vcf_file.fetch():
    # Get the start position (0-based for BED)
    start = rec.pos
    # For BED, the end position is the start + length of the reference allele
    end = start + len(rec.ref)-1
    # Create the BED format entry
    bed_entry = pybedtools.create_interval_from_list(["chr"+rec.chrom, start, end, rec.ref, ','.join(rec.alts)])
    # Append the entry to the list
    bed_entries.append(bed_entry)

# Create a BedTool object from the list of entries
vcf_bed = pybedtools.BedTool(bed_entries)
vcf_bed.saveas('/data/private/pdutta/Collab_data/Oliver_data/New_data/background_boundaries.SpliceAI.bed')

<BedTool(/data/private/pdutta/Collab_data/Oliver_data/New_data/background_boundaries.SpliceAI.bed)>

In [32]:
intersect_bed = acceptor_bed.intersect(vcf_bed, wa=True, wb=True)

In [33]:
for i, interval in enumerate(intersect_bed):
    if i < 5:
        print(interval)
    else:
        break

chr1	114780176	114780255	ENST00000060969.6	-	chr1	114780192	114780192	C	T

chr1	114779245	114779324	ENST00000060969.6	-	chr1	114779245	114779245	T	C

chr1	114776420	114776499	ENST00000060969.6	-	chr1	114776425	114776425	A	G

chr1	114776420	114776499	ENST00000060969.6	-	chr1	114776441	114776441	C	G

chr1	170719685	170719764	ENST00000239461.11	+	chr1	170719745	170719745	A	G



In [34]:
intersect_df = intersect_bed.to_dataframe()
intersect_df=intersect_df.drop(columns=['strand'])
new_columns = ['chr_name', 'Acceptor_start', 'Acceptor_end', 'Transcript_id', 'Strand', 'Variant_start', 'Variant_end',  'Reference_Nucleotide',  'Alternative_Nucleotides']
intersect_df.columns = new_columns
intersect_df

,chr_name,Acceptor_start,Acceptor_end,Transcript_id,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides
0,chr1,114780176,114780255,ENST00000060969.6,-,114780192,114780192,C,T
1,chr1,114779245,114779324,ENST00000060969.6,-,114779245,114779245,T,C
2,chr1,114776420,114776499,ENST00000060969.6,-,114776425,114776425,A,G
3,chr1,114776420,114776499,ENST00000060969.6,-,114776441,114776441,C,G
4,chr1,170719685,170719764,ENST00000239461.11,+,170719745,170719745,A,G
...,...,...,...,...,...,...,...,...,...
17975,chr22,22550295,22550374,ENST00000543184.5,-,22550312,22550312,C,T
17976,chr22,32538071,32538150,ENST00000619146.1,-,32538072,32538072,C,G
17977,chr22,32538071,32538150,ENST00000619146.1,-,32538085,32538085,T,C
17978,chr22,32528969,32529048,ENST00000619146.1,-,32528980,32528980,C,A


In [41]:
def seq2kmer(seq):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    k=6
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [50]:
# Add columns for reference and alternative sequences
intersect_df['Reference_Genome_Sequence'] = ''
intersect_df['Alternative_Genome_Sequence'] = ''
# Iterate through each row in the DataFrame
for idx, row in intersect_df.iterrows():
    # Fetch the reference sequence from the FASTA file
    acceptor_seq =  reference_fasta.fetch(row['chr_name'], row['Acceptor_start'] - 1, row['Acceptor_end'])
    
    variant_pos = row['Variant_start'] - row['Acceptor_start']
    
    # Construct the alternative sequence with the variant
    alt_seq = (acceptor_seq[:variant_pos] + row['Alternative_Nucleotides'] +
               acceptor_seq[variant_pos + 1:])
    
    # print(acceptor_seq)
    # print(alt_seq)
    # input()
    
    # Assign the sequences to the DataFrame
    intersect_df.at[idx, 'Reference_Genome_Sequence'] = acceptor_seq
    intersect_df.at[idx, 'Alternative_Genome_Sequence'] = alt_seq

reference_fasta.close()

merged_list = list(zip(intersect_df['Reference_Genome_Sequence'], intersect_df['Alternative_Genome_Sequence']))
merged_list = [item.upper() for tup in merged_list for item in tup]
#print(merged_list)
kmer_lst = list(map(seq2kmer, merged_list))
df_kmer = pd.DataFrame()
df_kmer = pd.DataFrame(kmer_lst, columns=['Sequence'])
df_kmer['Label'] = [1 if i % 2 == 0 else 0 for i in range(len(df_kmer))]

In [51]:
intersect_df

,chr_name,Acceptor_start,Acceptor_end,Transcript_id,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,Reference_Genome_Sequence,Alternative_Genome_Sequence
0,chr1,114780176,114780255,ENST00000060969.6,-,114780192,114780192,C,T,ATTTGGACATGTCCTTCATATCGGATGCATCCTCTTGATACTGGAC...,ATTTGGACATGTCCTTTATATCGGATGCATCCTCTTGATACTGGAC...
1,chr1,114779245,114779324,ENST00000060969.6,-,114779245,114779245,T,C,TCCAAAGCATCCTGGTGTTCCTCCAAGGAAATCCATAGCTCTGTCA...,CCCAAAGCATCCTGGTGTTCCTCCAAGGAAATCCATAGCTCTGTCA...
2,chr1,114776420,114776499,ENST00000060969.6,-,114776425,114776425,A,G,CTCCCATTTCACAGATTCTGTCAATCTGACTCTCAATTTCCTGTGA...,CTCCCGTTTCACAGATTCTGTCAATCTGACTCTCAATTTCCTGTGA...
3,chr1,114776420,114776499,ENST00000060969.6,-,114776441,114776441,C,G,CTCCCATTTCACAGATTCTGTCAATCTGACTCTCAATTTCCTGTGA...,CTCCCATTTCACAGATTCTGTGAATCTGACTCTCAATTTCCTGTGA...
4,chr1,170719685,170719764,ENST00000239461.11,+,170719745,170719745,A,G,CAGTGAATTTGGCTTCTTTTCATCATTGTGTTCTATTCCAGATGAC...,CAGTGAATTTGGCTTCTTTTCATCATTGTGTTCTATTCCAGATGAC...
...,...,...,...,...,...,...,...,...,...,...,...
17975,chr22,22550295,22550374,ENST00000543184.5,-,22550312,22550312,C,T,AGAGTTCTTCCGTAAATCCAGCACTTGAAGTTTCCACCTCCTGTGG...,AGAGTTCTTCCGTAAATTCAGCACTTGAAGTTTCCACCTCCTGTGG...
17976,chr22,32538071,32538150,ENST00000619146.1,-,32538072,32538072,C,G,GCCTGTGTTGGCCTTCCAGTTCCCAGAGATGGAGGTTCTCCTGTAC...,GGCTGTGTTGGCCTTCCAGTTCCCAGAGATGGAGGTTCTCCTGTAC...
17977,chr22,32538071,32538150,ENST00000619146.1,-,32538085,32538085,T,C,GCCTGTGTTGGCCTTCCAGTTCCCAGAGATGGAGGTTCTCCTGTAC...,GCCTGTGTTGGCCTCCCAGTTCCCAGAGATGGAGGTTCTCCTGTAC...
17978,chr22,32528969,32529048,ENST00000619146.1,-,32528980,32528980,C,A,CCACATGCTCTCCAATCAGCGGCATTGAGCTGTCCATTACCTGTGG...,CCACATGCTCTACAATCAGCGGCATTGAGCTGTCCATTACCTGTGG...


In [52]:
df_kmer

,Sequence,Label
0,ATTTGG TTTGGA TTGGAC TGGACA GGACAT GACATG ACAT...,1
1,ATTTGG TTTGGA TTGGAC TGGACA GGACAT GACATG ACAT...,0
2,TCCAAA CCAAAG CAAAGC AAAGCA AAGCAT AGCATC GCAT...,1
3,CCCAAA CCAAAG CAAAGC AAAGCA AAGCAT AGCATC GCAT...,0
4,CTCCCA TCCCAT CCCATT CCATTT CATTTC ATTTCA TTTC...,1
...,...,...
35955,GCCTGT CCTGTG CTGTGT TGTGTT GTGTTG TGTTGG GTTG...,0
35956,CCACAT CACATG ACATGC CATGCT ATGCTC TGCTCT GCTC...,1
35957,CCACAT CACATG ACATGC CATGCT ATGCTC TGCTCT GCTC...,0
35958,CCACAT CACATG ACATGC CATGCT ATGCTC TGCTCT GCTC...,1


In [53]:
intersect_df.to_csv("/data/private/pdutta/Collab_data/Oliver_data/New_data/Intersected_data/Acceptor/intersected_all_data.tsv", sep="\t", index=False)
df_kmer.to_csv("/data/private/pdutta/Collab_data/Oliver_data/New_data/DNABERT_data/Acceptor/dev.tsv", sep="\t", index=False)